In [1]:
import tensorflow as tf
from dataset_tf import VoxDataset, DataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
# from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
dataset = VoxDataset()

In [3]:
ids = dataset.get_ids()

107640


In [4]:
DATASET_SIZE = len(ids)

In [5]:
train_size = int(0.6 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size - val_size

In [6]:
print(train_size)
print(test_size)
print(val_size)

3229
1077
1076


In [7]:
np.random.shuffle(ids)

# Split the indices into training, validation, and test sets
train_indices = ids[:train_size]
val_indices = ids[train_size:train_size + val_size]
test_indices = ids[train_size + val_size:]

In [8]:
len(train_indices)

3229

In [9]:
train = VoxDataset(ids=train_indices)
test = VoxDataset(ids=test_indices)
val = VoxDataset(ids=val_indices)

In [10]:
type(train_indices)

list

In [11]:
print(len(train))
print(len(test))
print(len(val))

64580
21540
21520


In [12]:
train = DataGenerator(train, batch_size=32)
test = DataGenerator(test, batch_size=1)
val = DataGenerator(val, batch_size=32)

In [13]:
dataset[0][0].shape

2024-03-16 23:23:49.885054: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-16 23:23:49.885074: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-16 23:23:49.885080: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-16 23:23:49.885108: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-16 23:23:49.885123: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TensorShape([384])

In [14]:
dataset[0][1].shape

TensorShape([15069])

In [15]:
import tensorflow as tf

def AutoEncoder():
    # Define input layer
    input_layer = tf.keras.layers.Input(shape=(384,))
    
    # Encoder layers
    # encoder = tf.keras.layers.Dense(192, activation='ReLU')(input_layer)
    
    # Decoder layers
    decoder = tf.keras.layers.Dense(500, activation='ReLU')(input_layer)
    decoder = tf.keras.layers.Dense(768, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(1000, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(1536, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(2000, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(3072, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(5000, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(6144, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(8000, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(10000, activation='ReLU')(decoder)
    decoder = tf.keras.layers.Dense(12288, activation='ReLU')(decoder)
    decoder_output = tf.keras.layers.Dense(15069, activation='ReLU')(decoder)
    
    # Define the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=decoder_output)
    
    return model




In [16]:
model = AutoEncoder()


In [17]:
loss_fn = tf.keras.losses.MeanSquaredError()

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.00001)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define the number of epochs
num_epochs = 100
batch_size = 8

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Train the model using model.fit
model.fit(train, epochs=num_epochs, batch_size=batch_size, validation_data=val, 
          callbacks=[early_stopping]
          )

Epoch 1/100


2024-03-16 23:23:50.999117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


   9/2019 [..............................] - ETA: 9:04 - loss: 87958.7109 - accuracy: 0.0000e+00

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 500)               192500    
                                                                 
 dense_1 (Dense)             (None, 768)               384768    
                                                                 
 dense_2 (Dense)             (None, 1000)              769000    
                                                                 
 dense_3 (Dense)             (None, 1536)              1537536   
                                                                 
 dense_4 (Dense)             (None, 2000)              3074000   
                                                                 
 dense_5 (Dense)             (None, 3072)              614707

In [ ]:
loss, acc = model.evaluate(test)

21540/21540 [==============================] - 397s 18ms/step - loss: 33909.8945 - accuracy: 0.0355


In [ ]:
print(f"Accuracy: {acc*100:.4f}%\nLoss: {loss:.4f}")

Accuracy: 3.5515%
Loss: 33909.8945


In [ ]:
model.save('./model.h5', save_format='h5')

/Users/naweenk/miniforge3/envs/tf/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
y_pred = []

In [ ]:
for i in tqdm(test):
    pred = model.predict(i[0], verbose=0)*279.79984
    y_pred.append(VoxDataset.to_mesh_points(pred))



100%|██████████| 21540/21540 [15:05<00:00, 23.79it/s]


In [ ]:
import open3d as o3d
import numpy as np

for index, i in enumerate(y_pred):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(i)
    # o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(f"./Test/data{index}.ply", pcd)

In [ ]:
train[0][1][1]

array([205.21123 , 129.36862 ,  88.51011 , ..., 106.495544, 161.81659 ,
       149.57361 ], dtype=float32)

In [ ]:
y_pred[0][0]

array([  64.67786 , -139.8999  ,  -54.854828], dtype=float32)

In [ ]:
# import numpy as np
# from scipy.spatial import Delaunay
# import trimesh

# points = y_pred[0]

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# # Estimate normals for the point cloud
# pcd.estimate_normals()

# # Create a surface mesh from the point cloud using Poisson reconstruction
# mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# # Visualize the mesh (optional)
# o3d.visualization.draw_geometries([mesh])